# Introduction to Statistics with Python

```
Koen Plevoets
Last modified: 2020-12-12
```

# Class 5

## Chapter 8: Statistical inference with SciPy

All the functions for the basic statistical tests are in the submodule `stats` of **SciPy** (abbreviated as `scipy`).

In [ ]:
from scipy import stats

### 8.1 Tests for categorical data

We construct a frequency table of the columns `SmokeGroup` and `Mortality` (again):

In [ ]:
import pandas as pd
chol = pd.read_table('chol.txt')
chol_tab = pd.crosstab(chol.SmokeGroup, chol.Mortality)
print(chol_tab)

The **Pearson's test** for independence between rows and columns is done with the function `chi2_contingency()`. It accepts an array or a DataFrame as its argument, and it returns (a tuple of) four elements.

In [ ]:
chi2, p, dof, expected = stats.chi2_contingency(chol_tab)

In [ ]:
chi2

In [ ]:
p

In [ ]:
dof

In [ ]:
expected

**Fisher's Exact test** is an alternative for 2x2 tables and small samples. It is done with the function `fisher_exact()` which returns two elements.

In [ ]:
tea = pd.DataFrame( [[3, 1], [1, 3]], index = ['guessed: tea', 'guessed: milk'],
    columns = ['poured: tea', 'poured: milk'])
print(tea)

In [ ]:
OR, p = stats.fisher_exact(tea)

In [ ]:
OR

In [ ]:
p

### 8.2 Tests for continuous data

For continuous data we make the distinction between **one-sample tests** and **two-sample tests**.

One-sample tests

Which test to use depends on the **normality** of your data, so you have to check that first! The normality test can be done with `shapiro()`, and we will illustrate everything with the `iris` data.

In [ ]:
iris = pd.read_csv('iris.csv', sep=';') # If not read in yet
SpWi = iris['Sepal_Width']
SpWi.describe()

In [ ]:
W, p = stats.shapiro(SpWi)

In [ ]:
W

In [ ]:
p

There are other functions for testing normality in SciPy but will not cover them. An alternative way to check normality is graphically by means of the function `probplot()`.

In [ ]:
import matplotlib.pyplot as plt
stats.probplot(SpWi, plot = plt)
plt.show()

Now that we have checked normality, we can use a **one-sample t-test** for an hypothetical mean. The function is `ttest_1samp()` and you specify the hypothetical mean value as the second argument.

In [ ]:
stat, p = stats.ttest_1samp(SpWi, popmean = 4)    # Test for H0: mu = 4

In [ ]:
stat

In [ ]:
p

If your data are not normally distributed, then you can use the **Wilcoxon's signed-rank test**. This is the non-parametric alternative to the 1-sample t-test and the function for it is `wilcoxon()`. However, it always tests for `popmean = 0`, so you have **substract** your hypothetical value from your data.

In [ ]:
SpLe = iris['Sepal_Length']
W, p_shapiro = stats.shapiro(SpLe)

In [ ]:
W

In [ ]:
p_shapiro

In [ ]:
stat, p_wilcoxon = stats.wilcoxon(SpLe - 4)

In [ ]:
stat

In [ ]:
p_wilcoxon

Two-sample tests

For comparing the means of **two samples**, we make the distinction between **independent** and **paired** samples:

- **Independent samples** are the observations from two **different groups**, e.g. Species `versicolor` vs. `virginica`.
- **Paired samples** are two **measurements on the same set** of observations, e.g. `Sepal_Length` vs. `Petal_Length`.

For independent samples you need to check **two** assumptions:

- Check whether **both** samples are **normally distributed**.
- Check whether both samples are "homoscedastic", i.e. have **equal variances**.

In the `iris` data, the `Sepal_Width` variable seems normally distributed for both `versicolor` and `virginica`:

In [ ]:
SpWi_versicolor = iris.loc[iris.Species == 'versicolor', 'Sepal_Width']
SpWi_virginica = iris.loc[iris.Species == 'virginica', 'Sepal_Width']
W_versicolor, p_versicolor = stats.shapiro(SpWi_versicolor)
W_virginica, p_virginica = stats.shapiro(SpWi_virginica)

In [ ]:
p_versicolor

In [ ]:
p_virginica

If **both** samples are **normally distributed**, then you can check **equality of variances** with **Bartlett's test**. The function for Bartlett's test is `bartlett()` which returns two elements.

In [ ]:
stat_bartlett, p_bartlett = stats.bartlett(SpWi_versicolor, SpWi_virginica)

In [ ]:
stat_bartlett

In [ ]:
p_bartlett

If one of the samples **deviates from normality**, then you can use **Levene's test** for **equality of variances**. The function is `levene()` which also returns two elements.

In [ ]:
stat_levene, p_levene = stats.levene(SpWi_versicolor, SpWi_virginica)

In [ ]:
stat_levene

In [ ]:
p_levene

The check for equality of variances has repercussions for the comparison of the means, i.e. the **t-test**. The function for it is `ttest_ind()` which has a third argument `equal_var` which is by default `True`.

In [ ]:
stat_t, p_t = stats.ttest_ind(SpWi_versicolor, SpWi_virginica, equal_var = True)

In [ ]:
stat_t

In [ ]:
p_t

Not necessary:

In [ ]:
stats.ttest_ind(SpWi_versicolor, SpWi_virginica, equal_var = False)

If **one** of the samples is **not normally distributed**, then you can only use the t-test for **large samples** (i.e. > 100). Otherwise, you have to use the non-parametric **Mann-Whitney U test**, the function for which is `mannwhitneyu()`.

In [ ]:
PtWi_versicolor = iris.loc[iris.Species == 'versicolor', 'Petal_Width']
PtWi_virginica = iris.loc[iris.Species == 'virginica', 'Petal_Width']
W_versicolor, p_versicolor = stats.shapiro(PtWi_versicolor)
W_virginica, p_virginica = stats.shapiro(PtWi_virginica)

In [ ]:
p_versicolor

In [ ]:
p_virginica

The Mann-Whitney U test does not require a check for equality of variances:

In [ ]:
stat_U, p_U = stats.mannwhitneyu(PtWi_versicolor, PtWi_virginica)

In [ ]:
stat_U

In [ ]:
p_U

With **paired data** you only need to check for **normality** of both variables, **not** the **equality of variances**. We illustrate everything with the variables `Sepal_Length` and `Petal_Length` for the `versicolor` Species:

In [ ]:
SpLe_versicolor = iris.loc[iris.Species == 'versicolor', 'Sepal_Length']
PtLe_versicolor = iris.loc[iris.Species == 'versicolor', 'Petal_Length']
W_Sp, p_Sp = stats.shapiro(SpLe_versicolor)
W_Pt, p_Pt = stats.shapiro(PtLe_versicolor)

In [ ]:
p_Sp

In [ ]:
p_Pt

The function for the **paired t-test** is `ttest_rel()`.

In [ ]:
stat_paired, p_paired = stats.ttest_rel(SpLe_versicolor, PtLe_versicolor)

In [ ]:
stat_paired

In [ ]:
p_paired

If **one** of the samples is **not normally distributed**, then you can again only use the t-test for **large samples** (i.e. > 100). The alternative is to use the **Wilcoxon's Signed-Rank test** (but for this data it is actually not necessary):

In [ ]:
stat_wcx, p_wcx = stats.wilcoxon(SpLe_versicolor, PtLe_versicolor)

In [ ]:
stat_wcx

In [ ]:
p_wcx

The straightforward generalization of comparing two samples is comparing **more than two samples**. For such comparisons, there is again a parametric technique and a non-parametric alternative.

The parametric technique is **One-way ANOVA** and the function for it is `f_oneway()`. We illustrate it by comparing the `Sepal_Width` for all three `Species`:

In [ ]:
SpWi_setosa = iris.loc[iris.Species == 'setosa', 'Sepal_Width']
W_setosa, p_setosa = stats.shapiro(SpWi_setosa)
p_setosa

In [ ]:
SpWi_setosa.describe()

In [ ]:
SpWi_versicolor.describe()

In [ ]:
SpWi_virginica.describe()

In [ ]:
stat_oneway, p_oneway = stats.f_oneway(SpWi_setosa, SpWi_versicolor, SpWi_virginica)

In [ ]:
stat_oneway

In [ ]:
p_oneway

The non-parametric alternative is the **Kruskal-Wallis test** and the function for it is `kruskal()`.

In [ ]:
stat_KW, p_KW = stats.kruskal(SpWi_setosa, SpWi_versicolor, SpWi_virginica)

In [ ]:
stat_KW

In [ ]:
p_KW

### Exercises

15. Perform the following statistical tests:

  15.1 Test whether each of the three substances in the `substance` dataset depend on `Gender`. Do the same for `Race`.

  15.2 Test whether the cholesterol values are the same for the smokers and the non-smokers. **Hint**: you probably need to make two separate objects. Use indexes to do that. Motivate each step of your approach!

## Chapter 9: Statistical modelling with statsmodels

Statistical models such as **ANOVA** are usually fitted with the functionalities of the module **statsmodels**.

In [ ]:
import statsmodels.formula.api as smf
iris_oneway = smf.ols('Sepal_Width ~ Species', iris).fit()
print(iris_oneway.summary())

**Linear regression** has the same structure as ANOVA but the predictor is **numeric**.

In [ ]:
iris_lm = smf.ols('Sepal_Width ~ Petal_Width', iris).fit()
print(iris_lm.summary())

**Multiple linear regression** involves **more than one** predictor and potential **nonlinear** terms.

In [ ]:
iris_reg = smf.ols('Sepal_Width ~ Petal_Width + Sepal_Length + I(Petal_Width ** 2)', iris).fit()
print(iris_reg.summary())

Finally, ANCOVA is the name for a linear model involving the **interaction** between a numeric and a categorical predictor.

In [ ]:
iris_ancova = smf.ols('Sepal_Width ~ Petal_Width + Species + Petal_Width : Species', iris).fit()
print(iris_ancova.summary())

Abbreviated formula:

In [ ]:
iris_ancova = smf.ols('Sepal_Width ~ Petal_Width * Species', iris).fit()
print(iris_ancova.summary())

### Exercises

16. Fit the following statistical models:

  16.1 Fit a linear model of the variables in the `chol.txt` data with `Cholesterol` as the response variable. Compare various models.
  
  16.2 Fit linear models for each of the four subsets of Anscombe's Quartet, i.e. `I`, `II`, `III` and `IV`. Compare the four models! Anscombe's Quartet is a built-in dataset of the seaborn module:

In [ ]:
import seaborn as sns    # If not loaded yet
anscombe = sns.load_dataset('anscombe')